<a href="https://colab.research.google.com/github/Apolinar97/CS497_tweet_sentiment_analysis/blob/master/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#provide acess to drive.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#locate csv file and download/load it.
downloaded = drive.CreateFile({'id':'166vhh0NJH_DyhTMYTnt-aQKkN7RyO9F2'})
downloaded.GetContentFile('Tweets.csv') 

In [0]:
#import necessary modules
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
#clean data and conduct EDA
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer

df = pd.read_csv('Tweets.csv')
col_to_drop = ['tweet_id','airline_sentiment_confidence',
              'negativereason', 'negativereason_confidence', 'airline',
              'airline_sentiment_gold', 'name', 'negativereason_gold',
              'retweet_count','tweet_coord', 'tweet_created',
              'tweet_location', 'user_timezone']
df.drop(col_to_drop,axis=1, inplace=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def norm_tweet(tweet):
  tweet = tweet.lower()
  
  return tweet


In [7]:
#clean data and conduct EDA
clean_df = df
#convert neutral to postitive reviews b/c negative reviews outnumber postive reviews
clean_df.replace(to_replace='neutral',value='positive',inplace=True)
clean_df.head()

,airline_sentiment,text
0,positive,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,positive,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [0]:
try:
  %tensorflow_version 2.x
except:
  pass
  
import tensorflow as tf
